## Selenium
- 웹 페이지 자동화 테스트 도구
- 웹 페이지 제어 (클릭, 스크롤, 입력) 하는 도구로 사용

In [5]:
!pip install selenium

In [14]:
import time # 딜레이를 사용하기 위한 라이브러리

from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.keys import Keys # 키보드를 제어하는 라이브러리

In [21]:
# chromedriver.exe 파일을 제어하기 위해 driver 변수 저장
driver = webdriver.Chrome()

url = 'https://www.naver.com/'

# get(url) : 제어할 웹 페이지 띄우는 함수
driver.get(url)

In [10]:
# 띄워진 브라우저 종료
driver.quit()

In [9]:
# 검색창 요소에 접근 --> 보낼 검색어 입력
driver.find_element_by_css_selector('#query').send_keys('펭수\n')

In [3]:
driver.find_element_by_css_selector('#query').send_keys('펭수')
# 검색버튼 클릭
driver.find_element_by_css_selector('#search_btn').click()

In [8]:
# 스크롤 제어
# - 아래키, 스페이스바, end키, page down키
driver.find_element_by_css_selector('body').send_keys(Keys.ARROW_DOWN)

In [9]:
driver.find_element_by_css_selector('body').send_keys(Keys.SPACE)

In [13]:
driver.find_element_by_css_selector('body').send_keys(Keys.PAGE_DOWN)

In [10]:
driver.find_element_by_css_selector('body').send_keys(Keys.END)

## 횟수를 지정하여 스크롤 제어

In [25]:
for i in range(100) :
    driver.find_element_by_css_selector('body').send_keys(Keys.PAGE_DOWN)
    time.sleep(2)

## 한솥도시락 메뉴/가격 수집
- 메뉴, 가격을 csv파일로 저장

In [39]:
driver = webdriver.Chrome()

url = 'https://www.hsd.co.kr/menu/menu_list'

driver.get(url)

In [40]:
# '더보기' 버튼 3번 클릭하기
for i in range(3) :
    driver.find_element_by_css_selector('a.c_05').click()
    time.sleep(1) # 바로 다 요청하면 못할 수 있으니 텀을 준다

In [41]:
driver.page_source # res.text와 같이 HTML소스코드를 문자열로 반환

'<html lang="ko"><head>\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\n    <meta name="viewport" content="width=device-width,initial-scale=1.0,minimum-scale=1.0,maximum-scale=1.0,user-scalable=no">\n\n    <!-- SNS LINK -->\n    <meta property="og:type" content="website">\n    <meta property="og:title" content="">\n    <meta property="og:url" content="">\n    <meta property="og:image" content="">\n    <meta property="og:description" content="">\n    <!--// SNS LINK -->\n\n    <meta name="_csrf_parameter" content="_csrf"><meta name="_csrf_header" content="X-CSRF-TOKEN"><meta name="_csrf" content="c5b4d1ee-640a-4b46-ad49-f93e1fda9ace"><title>전체메뉴 &gt; 한솥 메뉴</title>\n\n    <link rel="icon" href="/assets/images/common/favicon.ico" type="image/x-icon">\n\n<link rel="stylesheet" type="text/css" href="/assets/css/ui.common.css">\n<link rel="stylesheet" type="text/css" href="/assets/css/swiper.min.css">\n<!--[

In [42]:
# 모든 제어가 끝난 후 bs로 변환
# 변환이 끝난 후 브라우저 종료

html = bs(driver.page_source, 'html.parser')
driver.quit()

한솥 메뉴, 가격 불러오기

In [43]:
menu_price = html.select('div.item-text')

In [59]:
menu_list = []
price_list = []
for i in range(len(menu_price)):
        menu_end = menu_price[i].text.find('격')
        
        menu_list.append(menu_price[i].text[0:menu_end-1])
        price_list.append(menu_price[i].text[menu_end+2:])

dataframe으로 만들고 csv 저장

In [52]:
from pandas import DataFrame

In [56]:
df = DataFrame({"메뉴":menu_list, "가격":price_list})
df

,메뉴,가격
0,"매화(치킨, 연어구이)","10,000원"
1,매화 (순살 고등어 간장구이),"10,000원"
2,진달래,"7,000원"
3,개나리(순살 고등어 간장구이),"8,000원"
4,생선까스도련님고기고기,"5,500원"
...,...,...
95,한솥 두부강된장소스,"1,800원"
96,한솥 감초볶음고추장소스,500원
97,볶음김치,400원
98,김치,300원


In [58]:
df.to_csv('한솥메뉴가격.csv', encoding = 'utf-8-sig')

## 멜론차트 곡명, 좋아요 수집
- '멜론_음원_정보.csv' 저장

In [60]:
driver = webdriver.Chrome()

url = 'https://www.melon.com/chart/day/index.htm'

driver.get(url)

In [61]:
html = bs(driver.page_source, 'html.parser')
driver.quit()

In [62]:
likes = html.select('span.cnt') # span.odd_span + span.cnt

In [72]:
like_list = []
for i in range(len(likes)):
    if i > 6 :
        like_tmp = likes[i].text.strip()
        like_list.append(like_tmp[4:])

In [75]:
songnames = html.select('div.ellipsis.rank01')

In [76]:
len(songnames)

100

In [78]:
song_list = []
for i in range(len(songnames)):
    song_list.append(songnames[i].text.strip())

In [79]:
df = DataFrame({"노래제목":song_list, "좋아요수":like_list})
df

,노래제목,좋아요수
0,Celebrity,"152,646"
1,밤하늘의 별을(2020),"114,169"
2,VVS (Feat. JUSTHIS) (Prod. GroovyRoom),"162,794"
3,Dynamite,"379,152"
4,이 밤을 빌려 말해요 (바른연애 길잡이 X 10CM),"28,044"
...,...,...
95,ON,"255,325"
96,너도 아는,"41,359"
97,"여백의 미 (Feat. Jessi, JUSTHIS) (Prod. GroovyRoom)","24,483"
98,나로 바꾸자 (duet with JYP),"20,029"


In [80]:
df.to_csv('멜론_음원_정보.csv', encoding = 'utf-8-sig', index = False)

## 스타벅스 음료 영양정보 수집
1. '사진으로보기' -> '영양정보로보기'를 클릭
2. bs로 변환
3. 음료명과 영양정보들을 수집
4. 스타벅스_음료_영양정보.csv로 저장

In [81]:
driver = webdriver.Chrome()

url = 'https://www.starbucks.co.kr/menu/drink_list.do'

driver.get(url)

In [82]:
driver.find_element_by_css_selector('a.a2').click()

In [83]:
html = bs(driver.page_source, 'html.parser')
driver.quit()

In [85]:
nutris = html.select('td')

In [98]:
len(nutris)

903

In [88]:
nutris[0].text

'나이트로 바닐라 크림'

In [94]:
menu_list = []
cal_list = []
sug_list = []
prot_list = []
nat_list = []
fat_list = []
caf_list = []

for i in range(len(nutris)):
    if i%7 == 0 :
        menu_list.append(nutris[i].text)
    if i%7 == 1 :
        cal_list.append(nutris[i].text)
    if i%7 == 2 :
        sug_list.append(nutris[i].text)
    if i%7 == 3 :
        prot_list.append(nutris[i].text)
    if i%7 == 4 :
        nat_list.append(nutris[i].text)
    if i%7 == 5 :
        fat_list.append(nutris[i].text)
    if i%7 == 6 :
        caf_list.append(nutris[i].text)

In [96]:
df = DataFrame({"메뉴":menu_list, "칼로리(Kcal)":cal_list, "당류(g)":sug_list, 
                "단백질(g)":prot_list, "나트륨(mg)":nat_list, "포화지방(g)":fat_list, "카페인(mg)":caf_list})
df

,메뉴,칼로리(Kcal),당류(g),단백질(g),나트륨(mg),포화지방(g),카페인(mg)
0,나이트로 바닐라 크림,75,10,1,20,2,245
1,나이트로 콜드 브루,5,0,0,5,0,245
2,돌체 콜드 브루,265,29,8,115,9,150
3,바닐라 크림 콜드 브루,125,11,3,58,6,150
4,제주 비자림 콜드 브루,340,44,10,115,8,105
...,...,...,...,...,...,...,...
124,퍼플베리 굿 190ML,100,24,0,15,0,0
125,한라봉주스 190ML,78,16,0.5,10,0,0
126,햇사과 주스 190ML,113,27,0.2,10,0,0
127,블루베리 요거트 190ML,155,18,5,70,3.6,0


In [97]:
df.to_csv('스타벅스_영양_정보.csv', encoding = 'utf-8-sig', index = False)

## 테이블 사용방법

In [105]:
import pandas as pd

In [111]:
driver = webdriver.Chrome()

url = 'https://www.starbucks.co.kr/menu/drink_list.do'

driver.get(url)

driver.find_element_by_css_selector('.a2').click()

html = bs(driver.page_source, 'html.parser')
driver.quit()

In [113]:
drink_info = html.select('table.coffeeInfo.mb60')

# 데이터 프레임의 사용할 컬럼명
cols = []

for col in drink_info[0].select('th'):
    cols.append(col.text)
    
# 데이터프레임에 저장할 행 리스트 생성
rows = []

# 모든 테이블에 접근하여 td요소의 데이터 접근
for info in drink_info: 
    for td in info.select('td'):
        rows.append(td.text)

# cols 리스트를 이용하여 빈 데이터프레임 생성
drink_df = pd.DataFrame(columns=cols)

# 데이터프레임에 한 행씩 저장
for i in range(0, len(rows), 7):
    drink_df.loc[i] = rows[i:i+7]
    
drink_df.to_csv('스타벅스_영양_정보2.csv', encoding = 'utf-8-sig', index = False)